In [1]:
import pandas as pd
import ipywidgets as widgets
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import datetime

In [2]:
PATH = pathlib.Path("./data")
data = pd.read_csv(PATH / "prepared_data.csv")
data["Date"] = pd.to_datetime(data["Date"])
data.index = data["Date"]
data.index.name = "DateIndex"

In [19]:
%matplotlib inline
%matplotlib widget
plt.close("all")
class App2:
    def __init__(self, dataset) -> None:
        self.colors = ["red", "green", "blue", "yellow"]
        self.dataset = dataset
        dates = self.dataset["Date"].dt.to_period('M').unique().to_timestamp()
        options = [(i.strftime('%b-%Y '), i) for i in dates]
        self.w_output_W_T = widgets.Output()
        self.w_output_W_D = widgets.Output()
        with self.w_output_W_T:
            self.fig_W_T, self.ax_W_T = plt.subplots(1, 1, constrained_layout=True)
            plt.show(self.fig_W_T)
        with self.w_output_W_D:
            self.fig_W_D, self.ax_W_D = plt.subplots(1, 1, constrained_layout=True)
            plt.show(self.fig_W_D)

        self.grid = widgets.GridspecLayout(5, 6)#, height='900px', width="900px")


        self.w_date_from = widgets.DatePicker(description='From',
                                            disabled=False,
                                            value=self.dataset["Date"].iloc[0].date(),
                                            continuous_update=False)        
        self.w_date_to = widgets.DatePicker(description='To',
                                          disabled=False,
                                          value=self.dataset["Date"].iloc[-1].date(),
                                          continuous_update=False)
        self.w_button_changed_date = widgets.Button(description="plot graph")
        self.w_description = widgets.Label(value="Select date range from to")
        self.w_information_status = widgets.Label(value="")
        self.w_information_status.style.text_color = "red"
        
        
        self.w_date_slider = widgets.SelectionRangeSlider(
            options=options,
            description='Date range',
            disabled=False,
            continuous_update=False,
            value=(dates[0], dates[-1]),
            layout=widgets.Layout(width="70%", height='auto'),
        )

        self.w_checkboxes = widgets.VBox(
            [widgets.Checkbox(value=False, description=note) for note in self.dataset["notes"].unique()]
        )
        self.w_button_clear_checkboxes = widgets.Button(description='Clear groups', disabled=False)

        self.w_right_side = widgets.VBox([
                                          widgets.HBox([self.w_date_from,
                                                        self.w_date_to]),
                                          self.w_button_changed_date,
                                          self.w_date_slider, 
                                          self.w_information_status
                                          ])
        self.w_left_side = widgets.VBox([
                                         self.w_checkboxes,
                                         self.w_button_clear_checkboxes,
                                         ])
    
        self.grid[1:, :3] = self.w_output_W_T
        self.grid[1:, 3:] = self.w_output_W_D

        self.grid[:1, :3] = self.w_left_side
        self.grid[:1, 3:] = self.w_right_side

        [checkbox.observe(self.checkboxes_handler) for checkbox in self.w_checkboxes.children]
        self.w_date_slider.observe(self.on_date_slider_changed, "value")
        self.w_button_changed_date.on_click(self.on_date_changed)
        self.w_button_clear_checkboxes.on_click(self.clear_checkboxes)

    def check_valid_date_range(self, date_range):
        date_to = date_range.stop
        date_from = date_range.start
        is_time_delta_positive = (date_to - date_from).total_seconds() > 0
        return is_time_delta_positive

    def on_date_changed(self, _):
        date_range = slice(self.w_date_from.value, self.w_date_to.value)
        self.replot_graph(date_range)

    def on_date_slider_changed(self, value):
        date_range = slice(value["new"][0], value["new"][-1])
        self.replot_graph(date_range)
    
    def checkboxes_handler(self, _):
        date_range = slice(self.w_date_from.value, self.w_date_to.value)
        self.replot_graph(date_range)

    def clear_checkboxes(self, _):
        for checkbox in self.w_checkboxes.children:
            checkbox.value = False

    def replot_graph(self, date_slice, **kwargs):
        y_min_max = (0, 5)
        x_min_max = (-5, 40)
        self.ax_W_D.cla()
        self.ax_W_T.cla()
        W_T_plot_kwargs = {"x": "Temp_avg",
                           "y": "Value (kWh)",
                           "ylim": y_min_max,
                           "xlim": x_min_max,
                           "ax": self.ax_W_T,
                           "xlabel": "Средняя температура, С",
                           "ylabel": "Потребление энергии (kWh)",
                           "title": "Зависимость энергопотребления от температуры",
                           "grid": True}
        if self.check_valid_date_range(date_slice):
            self.w_information_status.value = ""
            sliced_dataset = self.dataset.loc[date_slice]
            with self.w_output_W_D:
                sliced_dataset.plot(y="Value (kWh)",
                                                ax=self.ax_W_D,
                                                title="Зависимость энергопотребления от времени",
                                                xlabel="Время",
                                                ylabel="Потребление энергии (kWh)",
                                                grid=True,
                                                color="green",
                                                ylim=y_min_max,
                                                ).legend(loc="upper center")
            with self.w_output_W_T:
                true_counts = 0
                for idx, checkbox in enumerate(self.w_checkboxes.children):
                    if checkbox.value:
                        true_counts += 1
                        subdataset = sliced_dataset[sliced_dataset["notes"] == checkbox.description]
                        subdataset.plot.scatter(c = self.colors[idx],
                                                label=checkbox.description,
                                                **W_T_plot_kwargs
                                                ).legend(loc="upper center")
                if true_counts == 0:
                    sliced_dataset.plot.scatter(**W_T_plot_kwargs)

        else:
            self.w_information_status.value = "Incorrect date range"

    def run(self):
        return self.grid

app2 = App2(data)
app2.run()

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

Checkbox(value=True, description='weekday')
Checkbox(value=True, description='weekend')
Checkbox(value=True, description='vacation')
Checkbox(value=True, description='COVID_lockdown')
Checkbox(value=True, description='weekday')
Checkbox(value=True, description='weekend')
Checkbox(value=True, description='vacation')
Checkbox(value=True, description='COVID_lockdown')
Checkbox(value=True, description='weekday')
Checkbox(value=True, description='weekend')
Checkbox(value=True, description='vacation')
Checkbox(value=True, description='COVID_lockdown')
Checkbox(value=False, description='weekday')
Checkbox(value=False, description='weekend')
Checkbox(value=True, description='vacation')
Checkbox(value=True, description='COVID_lockdown')
Checkbox(value=True, description='weekday')
Checkbox(value=True, description='weekend')
Checkbox(value=True, description='vacation')
Checkbox(value=True, description='COVID_lockdown')
